# Step11 Train

# 11 trainは、ランダムから深堀をコントロール

 trainでは、epsilonを使って、緩やかにランダムの要素を減らしていきます。

## 11.1 コードを見ていきます。

In [ ]:
np.random.seed(seed=123)
env = EnvPuyopuyo()
agent = DQNAgent()


for epochs in range(CFG_ML.num_epochs):
  epsilon = CFG_ML.final_epsilon + (max(CFG_ML.num_decay_epochs - epochs, 0) * (
            CFG_ML.initial_epsilon - CFG_ML.final_epsilon) / CFG_ML.num_decay_epochs)

  board, puyo = env.reset()
  done = False
  final_score = 0
  final_pieces = 0

  while not done:
    u = random.random()
    random_action = u <= epsilon

    action = agent(board, puyo)

    if random_action:
      action_list = utils.create_action_list(board)
      if len(action_list):
        index = random.randint(0, len(action_list) - 1)
        action = action_list[index]

    next_board, puyo, reward, done = env.step(action)
    agent.update(board, action, reward, next_board, done)
    board = next_board
    final_score += reward
    final_pieces += 2
  
  if epochs % CFG_ML.sync_interval == 0:
    agent.sync_qnet()

  print("Epoch: {}/{}, Score: {}, pieces {}".format(
    epochs,
    CFG_ML.num_epochs,
    final_score,
    final_pieces))

  if epochs > 0 and epochs % CFG_ML.save_interval == 0:
    agent.save_model("{}/puyopuyo_{}".format(CFG_ML.saved_path, epochs))

agent.save_model("{}/puyopuyo".format(CFG_ML.saved_path))


## 11.2 解説するまでもなく、読んでそのままですが

* epsilon : ランダムにする比率です。0から1までの数値をランダムに出して、epsilonより小さければランダムに選択します。4000でランダム0になります。

25分くらいで終了します。